# Import Warnings

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings 
import seaborn as sns

# Load Data

In [ ]:
data=pd.read_csv("temperature-1.csv")
data

In [ ]:
data.drop(columns=["Unnamed: 0"],axis=1, inplace=True)
data

In [ ]:
data.info()

In [ ]:
# Convert data time column into a pandas date time object
data['Datetime']=pd.to_datetime(data['Datetime'], format= '%d-%m-%Y %H:%M')

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
# Set datetime object column as index --> That column is Datetime
data.set_index('Datetime', inplace=True)
data.head()

# Data Visualization

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(data)
plt.title("Hourly Temperature")
plt.xlabel("Datetime")
plt.ylabel("Temperature")
plt.grid(True)

# Stationary Check

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
# Augumented Dickey Fuller Test
dftest=adfuller(data['Hourly_Temp'])
dftest

In [ ]:
df_output=pd.Series(dftest[0:4], index=['Test Statistics', 'p-value','#Lags Used', 'No. of Observations Used'])

In [ ]:
for key, value in dftest[4].items():
    df_output['Critical value(%s)' %key]= value

In [ ]:
print("Augumented Dickey Fuller Test Result: ")
print(df_output)

In [ ]:
# Make it Stationary

# Differencing: Making it less than 0.06(Current -Previous)

In [ ]:
data['Temp_diff']= data['Hourly_Temp']- data['Hourly_Temp'].shift(1)# Informing python to skip 1st row

In [ ]:
data.head()

In [ ]:
data.dropna(inplace=True)
data.head()

In [ ]:
# Plot the difference column
plt.figure(figsize=(20,6))
plt.plot(data['Temp_diff'])
plt.title("Hourly Temperature Data Difference")
plt.xlabel("Datetime")
plt.ylabel("Temperature Difference")
plt.grid(True)

In [ ]:
# Run the ADF test
df_test2=adfuller(data['Temp_diff'])
df_test2

# Build the Arima model

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Build SARIMA model

In [ ]:
model= SARIMAX(data['Hourly_Temp'], order=(1,1,1), seasonal_order=(1,1,2,24))
result=model.fit(disp=False)

In [ ]:
# Predict the result
forecast_steps=24
forecast= results.get_forecast(steps=forecast_steps)

# Visualise - Date Range

In [ ]:
data_range= pd.data_range(start=data.index[-1], periods=forecast_steps+1, freq='H')
data_range

In [ ]:
forecast_values=forecast.predict_mean
forecast_conf_int=forecast.conf_int()

In [ ]:
forecast_df=pd.DataFrame(
    {
        'Forecast': forecast_values,
        'Lower_CI': forecast_conf_int.iloc[:,0],
        'Upper_CI': forecast_conf_int.iloc[:,1],
        'index': data_range[1:]
    } 
)    

In [ ]:
forecast_df.head()